# U-Net++ Hyperparameter Tuning

In [1]:
import sys
import os

# Add the parent directory to the Python path
parent_dir = os.path.abspath(os.path.join('../..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [3]:
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import ParameterSampler
import torch.nn.functional as F
from torchvision import transforms
from models import NUM_CLASSES, METAINFO

# Parameters
IMG_SIZE = 512
BATCH_SIZE = 8
NUM_EPOCHS = 5

In [5]:
class CombinedSegmentationLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0, boundary_weight=1.0, dice_weight=1.0):
        super(CombinedSegmentationLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.boundary_weight = boundary_weight
        self.dice_weight = dice_weight

    def forward(self, inputs, targets):
        # Multi-class Cross Entropy Loss
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        # Varifocal Loss component
        pred_prob = F.softmax(inputs, dim=1)
        pt = pred_prob.gather(1, targets.unsqueeze(1)).squeeze(1)
        focal_weight = self.alpha * (1 - pt).pow(self.gamma)
        vfl_loss = focal_weight * ce_loss

        # Dice Loss component
        smooth = 1e-6
        inputs_softmax = F.softmax(inputs, dim=1)
        targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[1]).permute(0, 3, 1, 2).float()
        intersection = (inputs_softmax * targets_one_hot).sum(dim=(2, 3))
        dice_loss = 1 - (2 * intersection + smooth) / (inputs_softmax.sum(dim=(2, 3)) + targets_one_hot.sum(dim=(2, 3)) + smooth)

        # Combine losses
        combined_loss = vfl_loss.mean() + self.dice_weight * dice_loss.mean()
        return combined_loss


In [6]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_prob=0.1):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_prob),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_prob)
        )

    def forward(self, x):
        return self.conv(x)

class UNetPlusPlus(nn.Module):
    def __init__(self, num_classes, deep_supervision=True, dropout_prob=0.1):
        super(UNetPlusPlus, self).__init__()
        self.deep_supervision = deep_supervision

        nb_filter = [32, 64, 128, 256, 512]

        self.pool = nn.MaxPool2d(2, 2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = ConvBlock(3, nb_filter[0], dropout_prob)
        self.conv1_0 = ConvBlock(nb_filter[0], nb_filter[1], dropout_prob)
        self.conv2_0 = ConvBlock(nb_filter[1], nb_filter[2], dropout_prob)
        self.conv3_0 = ConvBlock(nb_filter[2], nb_filter[3], dropout_prob)
        self.conv4_0 = ConvBlock(nb_filter[3], nb_filter[4], dropout_prob)

        self.conv0_1 = ConvBlock(nb_filter[0]+nb_filter[1], nb_filter[0], dropout_prob)
        self.conv1_1 = ConvBlock(nb_filter[1]+nb_filter[2], nb_filter[1], dropout_prob)
        self.conv2_1 = ConvBlock(nb_filter[2]+nb_filter[3], nb_filter[2], dropout_prob)
        self.conv3_1 = ConvBlock(nb_filter[3]+nb_filter[4], nb_filter[3], dropout_prob)

        self.conv0_2 = ConvBlock(nb_filter[0]*2+nb_filter[1], nb_filter[0], dropout_prob)
        self.conv1_2 = ConvBlock(nb_filter[1]*2+nb_filter[2], nb_filter[1], dropout_prob)
        self.conv2_2 = ConvBlock(nb_filter[2]*2+nb_filter[3], nb_filter[2], dropout_prob)

        self.conv0_3 = ConvBlock(nb_filter[0]*3+nb_filter[1], nb_filter[0], dropout_prob)
        self.conv1_3 = ConvBlock(nb_filter[1]*3+nb_filter[2], nb_filter[1], dropout_prob)

        self.conv0_4 = ConvBlock(nb_filter[0]*4+nb_filter[1], nb_filter[0], dropout_prob)

        if self.deep_supervision:
            self.final1 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final2 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final3 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
            self.final4 = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)
        else:
            self.final = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)

    def forward(self, input):
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1))

        if self.deep_supervision:
            output1 = self.final1(x0_1)
            output2 = self.final2(x0_2)
            output3 = self.final3(x0_3)
            output4 = self.final4(x0_4)
            return [output1, output2, output3, output4]
        else:
            output = self.final(x0_4)
            return output

In [11]:
def load_and_sample_data(file_path, sample_fraction=1):
    df = pd.read_csv(file_path)
    sampled_df = df.sample(frac=sample_fraction, random_state=42)
    return sampled_df

train_df = load_and_sample_data('../../dataset/train_sample_30.csv')
val_df = load_and_sample_data('../../dataset/val_sample_30.csv')
test_df = load_and_sample_data('../../dataset/test.csv')

imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

class WildScene(Dataset):
    def __init__(self, df, img_size, num_classes, is_train=False):
        self.df = df
        self.img_size = img_size
        self.num_classes = num_classes
        self.is_train = is_train

        # Training transforms with data augmentation
        self.train_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(15),
            transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            transforms.ToTensor(),
            transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
        ])

        # Evaluation transforms without data augmentation
        self.eval_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = cv2.imread('../../' + row['im_path'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = cv2.imread('../../' + row['label_path'], cv2.IMREAD_GRAYSCALE)

        if self.is_train:
            # Apply training transforms
            seed = torch.randint(0, 2**32, (1,)).item()
            torch.manual_seed(seed)
            img = self.train_transform(img)
            
            torch.manual_seed(seed)
            label = transforms.functional.to_tensor(
                transforms.functional.resize(
                    transforms.ToPILImage()(label), 
                    (self.img_size, self.img_size)
                )
            )
        else:
            # Apply evaluation transforms
            img = self.eval_transform(img)
            label = transforms.functional.to_tensor(
                transforms.functional.resize(
                    transforms.ToPILImage()(label), 
                    (self.img_size, self.img_size)
                )
            )
        label = label.squeeze().long()
        return img, label

In [12]:
# Create datasets with appropriate transforms
train_dataset = WildScene(train_df, IMG_SIZE, NUM_CLASSES, is_train=True)
val_dataset = WildScene(val_df, IMG_SIZE, NUM_CLASSES, is_train=False)
test_dataset = WildScene(test_df, IMG_SIZE, NUM_CLASSES, is_train=False)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [13]:
def train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs, early_stop_epochs=3):
    best_val_loss = float('inf')
    train_losses = []
    val_losses = []
    scaler = GradScaler()

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.cuda(), labels.cuda()
            optimizer.zero_grad()

            with autocast():
                outputs = model(images)
                if isinstance(outputs, list):
                    loss = sum([criterion(output, labels) for output in outputs]) / len(outputs)
                else:
                    loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
                images, labels = images.cuda(), labels.cuda()
                with autocast():
                    outputs = model(images)
                    if isinstance(outputs, list):
                        loss = sum([criterion(output, labels) for output in outputs]) / len(outputs)
                    else:
                        loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')

        # Early stopping check
        if epoch >= early_stop_epochs and val_losses[-early_stop_epochs] < val_loss:
            print(f"Early stopping at epoch {epoch+1}")
            break

    print("Training complete.")
    return model, train_losses, val_losses

In [14]:
def random_search(param_distributions, n_iter=10, initial_epochs=3, full_epochs=10):
    param_list = list(ParameterSampler(param_distributions, n_iter=n_iter))
    
    best_model = None
    best_val_loss = float('inf')
    best_params = None
    
    for i, params in enumerate(param_list):
        print(f"\nTrying combination {i+1}/{len(param_list)}:")
        print(params)

        model = UNetPlusPlus(num_classes=NUM_CLASSES, deep_supervision=True, dropout_prob=params['dropout_prob']).cuda()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
        criterion = CombinedSegmentationLoss(alpha=params['focal_alpha'], gamma=params['focal_gamma'], boundary_weight=params['boundary_weight'], dice_weight=params['dice_weight'])
        
        try:
            # Initial evaluation phase
            trained_model, _, val_losses = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=initial_epochs)
            
            if val_losses[-1] < best_val_loss:
                best_val_loss = val_losses[-1]
                best_model = trained_model
                best_params = params
                print(f"New best model found with initial validation loss: {best_val_loss}")

                # Full evaluation phase
                trained_model, _, val_losses = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=full_epochs)
                if val_losses[-1] < best_val_loss:
                    best_val_loss = val_losses[-1]
                    best_model = trained_model
                    best_params = params
                    print(f"New best model found with full validation loss: {best_val_loss}")
        except RuntimeError as e:
            print(f"Error during training: {str(e)}")
            continue
    
    return best_model, best_params

In [8]:
param_distributions = {
    'dropout_prob': [0.1, 0.2, 0.3],
    'learning_rate': [5e-4, 1e-3],
    'focal_alpha': [0.25, 0.5, 0.75],
    'focal_gamma': [1, 2, 3],
    'boundary_weight': [0.5, 1.0, 1.5],
    'dice_weight': [0.5, 1, 1.5]
}
best_model, best_params = random_search(param_distributions, n_iter=20, initial_epochs=3, full_epochs=10)
print("Best parameters:", best_params)

# Train the final model with the best parameters
final_model = UNetPlusPlus(num_classes=NUM_CLASSES, deep_supervision=True, dropout_prob=best_params['dropout_prob']).cuda()
final_optimizer = optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
final_criterion = CombinedSegmentationLoss(alpha=best_params['focal_alpha'], gamma=best_params['focal_gamma'], boundary_weight=best_params['boundary_weight'], dice_weight=best_params['dice_weight'])

trained_model, train_losses, val_losses = train_model(final_model, train_loader, val_loader, final_optimizer, final_criterion, num_epochs=NUM_EPOCHS)



Trying combination 1/20:
{'learning_rate': 0.0005, 'focal_gamma': 1, 'focal_alpha': 0.75, 'dropout_prob': 0.2, 'dice_weight': 1, 'boundary_weight': 0.5}


Validation Epoch 1/3: 100%|██████████| 65/65 [02:04<00:00,  1.92s/it]


Epoch 1/3, Train Loss: 2.4426, Val Loss: 2.0929


Validation Epoch 2/3: 100%|██████████| 65/65 [01:29<00:00,  1.37s/it]


Epoch 2/3, Train Loss: 1.6883, Val Loss: 1.4044


...

Epoch 3/5, Train Loss: 0.4948, Val Loss: 0.4841


Validation Epoch 4/5: 100%|██████████| 65/65 [01:30<00:00,  1.39s/it]


Epoch 4/5, Train Loss: 0.4837, Val Loss: 0.4788


Validation Epoch 5/5: 100%|██████████| 65/65 [01:30<00:00,  1.39s/it]

Epoch 5/5, Train Loss: 0.4797, Val Loss: 0.4770
Training complete.
